# Wiener SBND Closure Test for 1mu1p Measurement Variables

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib as mpl
from os import path
import sys
import uproot

from wienersvd import *
from unfolding_inputs import *

# import dunestyle.matplotlib as dunestyle
plt.style.use("presentation.mplstyle")

In [2]:
save_fig = False
save_fig_dir = "./plots/wiener_svd/1mu1p"

In [3]:
cmap = mpl.cm.viridis
norm = mpl.colors.Normalize(vmin=0.0, vmax=1.0)

# load dataframes

In [4]:
def get_n_split(file):
    this_split_df = pd.read_hdf(file, key="split")
    this_n_split = this_split_df.n_split.iloc[0]
    return this_n_split

def print_keys(file):
    with pd.HDFStore(file, mode='r') as store:
        keys = store.keys()       # list of all keys in the file
        print("Keys:", keys)

In [5]:
def load_dfs(file, keys2load):
    out_df_dict = {}
    this_n_keys = get_n_split(file)
    n_concat = min(n_max_concat, this_n_keys)
    for key in keys2load:
        dfs = []  # collect all splits for this key
        for i in range(n_concat):
            this_df = pd.read_hdf(file, key=f"{key}_{i}")
            dfs.append(this_df)
        out_df_dict[key] = pd.concat(dfs, ignore_index=False)

    return out_df_dict

In [ ]:
## -- MC study
mc_file = "/exp/sbnd/data/users/munjung/xsec/2025B/MCP_gump_new.df"
mc_split_df = pd.read_hdf(mc_file, key="split")
mc_n_split = get_n_split(mc_file)
print("mc_n_split: %d" %(mc_n_split))
print_keys(mc_file)

In [7]:
n_max_concat = 2

mc_keys2load = ['evt', 'hdr', 'mcnuwgtslim']
mc_dfs = load_dfs(mc_file, mc_keys2load)

mc_evt_df = mc_dfs['evt']
mc_hdr_df = mc_dfs['hdr']
mc_nu_df = mc_dfs['mcnuwgtslim']

In [ ]:
# TODO: move to makedf.py
# genie multisims
genie_knobs = ['GENIEReWeight_SBN_v1_multisim_ZExpAVariationResponse',
               'GENIEReWeight_SBN_v1_multisim_CCRESVariationResponse',
               'GENIEReWeight_SBN_v1_multisim_NCRESVariationResponse',
               'GENIEReWeight_SBN_v1_multisim_DISBYVariationResponse',
               'GENIEReWeight_SBN_v1_multisim_FSI_pi_VariationResponse',
               'GENIEReWeight_SBN_v1_multisim_FSI_N_VariationResponse',
               'GENIEReWeight_SBN_v1_multisim_NCELVariationResponse']

for uidx in range(100):
    for kidx, knob in enumerate(genie_knobs):
        if kidx == 0:
            mc_evt_df[("GENIE", "univ_{}".format(uidx), "", "", "", "", "", "")] = mc_evt_df[(knob, "univ_{}".format(uidx), "", "", "", "", "", "")]
            mc_nu_df[("GENIE", "univ_{}".format(uidx), "")] = mc_nu_df[(knob, "univ_{}".format(uidx), "")]
        else:
            mc_evt_df[("GENIE", "univ_{}".format(uidx), "", "", "", "", "", "")] *= mc_evt_df[(knob, "univ_{}".format(uidx), "", "", "", "", "", "")]
            mc_nu_df[("GENIE", "univ_{}".format(uidx), "")] *= mc_nu_df[(knob, "univ_{}".format(uidx), "")]

# POT

In [ ]:
## total pot
mc_tot_pot = mc_hdr_df['pot'].sum()
print("mc_tot_pot: %.3e" %(mc_tot_pot))

target_pot = 1e20
mc_pot_scale = target_pot / mc_tot_pot
print("mc_pot_scale: %.3e" %(mc_pot_scale))
mc_pot_scale = 1.

mc_evt_df["pot_weight"] = mc_pot_scale * np.ones(len(mc_evt_df))

# Constants

In [ ]:
# TODO: z-dependence?
# flux file, units: /m^2/10^6 POT 
# 50 MeV bins
fluxfile = "/exp/sbnd/data/users/munjung/flux/sbnd_original_flux.root"
flux = uproot.open(fluxfile)
print(flux.keys())

# numu flux
numu_flux = flux["flux_sbnd_numu"].to_numpy()
bin_edges = numu_flux[1]
flux_vals = numu_flux[0]

plt.hist(bin_edges[:-1], bins=bin_edges, weights=flux_vals, histtype="step")
plt.xlabel("E [GeV]")
plt.ylabel("Flux [/m$^{2}$/10$^{6}$ POT]")
plt.title("SBND $\\nu_\\mu$ Flux")
plt.show()

# get integrated flux
integrated_flux = flux_vals.sum()
integrated_flux /= 1e4 # to cm2
integrated_flux = integrated_flux * mc_tot_pot / 1e6 # POT
print("Integrated flux: %.3e" % integrated_flux)

In [ ]:
rho = 1.3836  #g/cm3, liquid Ar density
N_A = 6.02214076e23 # Avogadro’s number
M_Ar = 39.95 # g, molar mass of argon
V_sbnd = 2 * 360 * 175 * 440 # cm3, the active volume of the detector # TODO: get exact value
NTargets = rho * V_sbnd * N_A / M_Ar
print("# of targets: ", NTargets)

In [ ]:
# set to 1 for event rates
xsec_unit = 1 / (integrated_flux * NTargets)

xsec_unit = 1
print("xsec unit: ", xsec_unit)

# Selection Utils

In [12]:
def InFV(data): # cm
    xmin = -190.
    ymin = -190.
    zmin = 10.
    xmax = 190.
    ymax =  190.
    zmax =  450.
    return (np.abs(data.x) > 10) & (np.abs(data.x) < 190) & (data.y > ymin) & (data.y < ymax) & (data.z > zmin) & (data.z < zmax)


def IsNu(df):
    return ~df.pdg.isna()


def Signal(df): # definition                                                                                                                                                                                                                                                                         
    is_fv = InFV(df.position)
    is_1mu1p0pi = (df.nmu_27MeV == 1) & (df.npi_30MeV == 0) & (df.np_50MeV == 1) & (df.np_20MeV == 1) & (df.npi0 == 0) & (df.mu.genE > 0.25)
    return is_fv & is_1mu1p0pi

In [13]:
def get_int_category(df):
    is_notnu = ~IsNu(df)
    is_nu_outfv = IsNu(df) & ~InFV(df.position)
    is_signal = Signal(df)
    is_other_nu_infv = IsNu(df) & InFV(df.position) & ~Signal(df)

    nuint_categ = pd.Series(8, index=df.index)
    nuint_categ[is_notnu] = -1  # not nu
    nuint_categ[is_nu_outfv] = 0  # nu out of FV
    nuint_categ[is_signal] = 1    # nu in FV, signal
    nuint_categ[is_other_nu_infv] = 2  # nu in FV, not signal

    return nuint_categ


# signal need to come first for below code to work
mode_list = [1, 2, 0, -1]
mode_labels = ["Signal", "Non-sig. FV Nu", "Non FV Nu", "Not Nu"]
colors = ['#d62728',  # Red            
          '#1f77b4',  # Blue
          '#ff7f0e',
          '#7f7f7f']  # Gray

In [ ]:
# ccqe selection

# in FV
print(InFV(mc_evt_df.slc.vertex).value_counts())
mc_evt_df = mc_evt_df[InFV(mc_evt_df.slc.vertex)]

# mu length cut
mc_evt_df = mc_evt_df[mc_evt_df.mu.pfp.trk.len > 50]

# mu chi2 cut 
mc_evt_df = mc_evt_df[(mc_evt_df.mu.pfp.trk.chi2pid.I2.chi2_muon > 0) & (mc_evt_df.mu.pfp.trk.chi2pid.I2.chi2_muon < 25) & (mc_evt_df.mu.pfp.trk.chi2pid.I2.chi2_proton > 100)]
mc_evt_df = mc_evt_df[(mc_evt_df.p.pfp.trk.chi2pid.I2.chi2_muon > 0) & (mc_evt_df.p.pfp.trk.chi2pid.I2.chi2_muon < 90)]

# 1p0pi
twoprong_cut = (np.isnan(mc_evt_df.other_shw_length) & np.isnan(mc_evt_df.other_trk_length))
mc_evt_df = mc_evt_df[twoprong_cut]

In [ ]:
# TODO: move this to makedf.py
mc_evt_df.loc[:, ("mu","pfp","trk","truth","p","totp","","")] = np.sqrt(mc_evt_df.mu.pfp.trk.truth.p.genp.x**2 + mc_evt_df.mu.pfp.trk.truth.p.genp.y**2 + mc_evt_df.mu.pfp.trk.truth.p.genp.z**2)
mc_nu_df.loc[:, ('mu','totp','')] = np.sqrt(mc_nu_df.mu.genp.x**2 + mc_nu_df.mu.genp.y**2 + mc_nu_df.mu.genp.z**2)

mc_evt_df.loc[:, ("mu","pfp","trk","truth","p","dir","x","")] = mc_evt_df.mu.pfp.trk.truth.p.genp.x/mc_evt_df.mu.pfp.trk.truth.p.totp
mc_evt_df.loc[:, ("mu","pfp","trk","truth","p","dir","y","")] = mc_evt_df.mu.pfp.trk.truth.p.genp.y/mc_evt_df.mu.pfp.trk.truth.p.totp
mc_evt_df.loc[:, ("mu","pfp","trk","truth","p","dir","z","")] = mc_evt_df.mu.pfp.trk.truth.p.genp.z/mc_evt_df.mu.pfp.trk.truth.p.totp

mc_nu_df.loc[:, ("mu","dir","x")] = mc_nu_df.mu.genp.x/mc_nu_df.mu.totp
mc_nu_df.loc[:, ("mu","dir","y")] = mc_nu_df.mu.genp.y/mc_nu_df.mu.totp
mc_nu_df.loc[:, ("mu","dir","z")] = mc_nu_df.mu.genp.z/mc_nu_df.mu.totp

In [ ]:
# classify events into categories
mc_evt_df.loc[:,'nuint_categ'] = get_int_category(mc_evt_df)
mc_nu_df.loc[:,'nuint_categ'] = get_int_category(mc_nu_df)

print(mc_evt_df.nuint_categ.value_counts())
print(mc_nu_df.nuint_categ.value_counts()) # won't have -1 because nudf is all nu events

# Closure test 
- use MC signal as fake data

In [17]:
# muon momentum
var_save_name = "muon-p"

var_labels = [r"$\mathrm{P_\mu~[GeV/c]}$", 
              r"$\mathrm{P_\mu^{reco.}~[GeV/c]}$",  # reco
              r"$\mathrm{P_\mu^{true}~[GeV/c]}$"]  # true

bins = np.linspace(0.2, 2, 6)

var_evt_reco_col = ('mu', 'pfp', 'trk', 'P', 'p_muon', '', '', '')
var_evt_truth_col = ('mu', 'pfp', 'trk', 'truth', 'p', 'totp', '', '')
var_nu_col = ('mu', 'totp', '')

# # muon dir z
# var_save_name = "muon-dir_z"

# var_labels = [r"$\mathrm{cos(\theta_\mu)}$",
#               r"$\mathrm{cos(\theta_\mu^{reco.})}$",
#               r"$\mathrm{cos(\theta_\mu^{true})}$"]

# bins = np.linspace(-1, 1, 6)

# var_evt_reco_col = ('mu', 'pfp', 'trk', 'dir', 'z', '', '', '')
# var_evt_truth_col = ('mu', 'pfp', 'trk', 'truth', 'p', 'dir', 'z', '')
# var_nu_col = ('mu', 'dir', 'z')

np.clip is for including underflow events into the first bin and overflow evetns into the last bin

In [18]:
# Total MC reco muon momentum: for fake data
eps = 1e-8
var_total_mc = mc_evt_df[var_evt_reco_col]
var_total_mc = np.clip(var_total_mc, bins[0], bins[-1] - eps)

# mc_evt_df divided into mode for subtraction from data in futre
# first item in list is the signal
mc_evt_df_divided = [mc_evt_df[mc_evt_df.nuint_categ == mode]for mode in mode_list]

# Reco muon momentum for each 'nuint_categ' for stack plot and subtraction from the fake data
var_per_nuint_categ_mc = [mc_evt_df[mc_evt_df.nuint_categ == mode][var_evt_reco_col]for mode in mode_list]
var_per_nuint_categ_mc = [s.clip(bins[0], bins[-1] - eps) for s in var_per_nuint_categ_mc]
weights_per_categ = [mc_evt_df.loc[mc_evt_df.nuint_categ == mode, 'pot_weight'] for mode in mode_list]

# for response matrix
# Signal event's reco muon momentum after the event selection
var_signal = mc_evt_df[mc_evt_df.nuint_categ == 1][var_evt_reco_col]
var_signal = np.clip(var_signal, bins[0], bins[-1] - eps)
weight_signal = mc_evt_df.loc[mc_evt_df.nuint_categ == 1, 'pot_weight']

# Signal event's true muon momentum after the event selection
true_var_signal_sel = mc_evt_df[mc_evt_df.nuint_categ == 1][var_evt_truth_col]
true_var_signal_sel = np.clip(true_var_signal_sel, bins[0], bins[-1] - eps)
weight_true_signal = mc_evt_df.loc[mc_evt_df.nuint_categ == 1, 'pot_weight']

# for efficiency vector
# Signal event's true muon momentum without event selection
var_truth_signal = mc_nu_df[mc_nu_df.nuint_categ == 1][var_nu_col]
var_truth_signal = np.clip(var_truth_signal, bins[0], bins[-1] - eps)
weight_truth_signal = np.full_like(var_truth_signal, mc_pot_scale, dtype=float)

Draw true (before event selection) and reco (after event selection) muon momentum distributions of signal events.
Print entries for double check.

In [ ]:
true_signal, _, _ = plt.hist(var_truth_signal, bins=bins, weights=weight_truth_signal, histtype="step", label="True Signal")
reco_signal_sel, _, _ = plt.hist(var_signal, bins=bins, weights=weight_signal, histtype="step", label="Reco Selected Signal")
true_signal_sel, _, _ = plt.hist(true_var_signal_sel, bins=bins, weights=weight_signal, histtype="step", label="True Selected Signal")
print(true_signal)
print(reco_signal_sel)
print(true_signal_sel)
plt.legend()
plt.ylabel("Events")
plt.xlim(bins[0], bins[-1])
plt.xlabel(var_labels[0])
if save_fig:
    plt.savefig("{}/{}-sel_event_rates.pdf".format(save_fig_dir, var_save_name), bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
mc_stack, bins, _ = plt.hist(var_per_nuint_categ_mc,
                            bins=bins,
                            weights=weights_per_categ,
                            stacked=True,
                            color=colors,
                            label=mode_labels,
                            edgecolor='none',
                            linewidth=0,
                            density=False,
                            histtype='stepfilled')

totmc, bin_edges = np.histogram(var_total_mc, bins=bins, weights=mc_evt_df.pot_weight)
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

# use MC as fake data for closure test
fake_data = totmc
fake_data_err = np.sqrt(totmc)
# plt.step(bin_edges[:-1], fake_data, where='post', label="Data")  # line
plt.errorbar(bin_centers, fake_data, yerr=fake_data_err, fmt='o', color='black')  # error bars

accum_sum = [np.sum(data) for data in mc_stack]
accum_sum = [0.] + accum_sum
total_sum = accum_sum[-1]
individual_sums = [accum_sum[i + 1] - accum_sum[i] for i in range(len(accum_sum) - 1)]
fractions = [(count / total_sum) * 100 for count in individual_sums]
legend_labels = [f"{label} ({frac:.1f}%)" for label, frac in zip(mode_labels[::-1], fractions[::-1])]
legend_labels.append("Fake Data")
plt.legend(legend_labels, loc='upper left', fontsize=10, frameon=False, ncol=3, bbox_to_anchor=(0.05, 0.98))

plt.xlim(bins[0], bins[-1])
plt.xlabel(var_labels[1])
plt.ylim(0., 1.2 * fake_data.max())
plt.ylabel("Events")

if save_fig:
    plt.savefig("{}/{}-fake_data.pdf".format(save_fig_dir, var_save_name), bbox_inches='tight')
plt.show()

# response matrix

In [ ]:
bins_2d = bins# = [np.array([0.2, 2]), np.array([0.2, 2])] # commented out lines for 1 bin MC closure test
reco_vs_true = get_smear_matrix(true_var_signal_sel, var_signal, bins_2d, var_labels=var_labels)
eff = get_eff(reco_vs_true, true_signal)
print("eff")
print(eff)
Response = get_response_matrix(reco_vs_true, eff, bins, var_labels=var_labels)

# Covariance

## Flux

In [ ]:
bin_centers = (bins[:-1] + bins[1:])/2
univ_events = []

# get background subtracted event rate for fake data
nominal_event = fake_data
for this_mc_evt_df in mc_evt_df_divided[1:]:  # first item is the signal
    this_var = this_mc_evt_df[var_evt_reco_col]
    this_var = np.clip(this_var, bins[0], bins[-1] - eps)
    this_n, bins = np.histogram(this_var, bins=bins)
    nominal_event = nominal_event - this_n

# get syst universes on background subtracted event rate for fake data
n_univ_flux = 1000
for uidx in range(n_univ_flux):
    this_signal = fake_data # before subtraction
    for this_mc_evt_df in mc_evt_df_divided[1:]: # first item is the signal
        weights = this_mc_evt_df["Flux"]["univ_{}".format(uidx)].copy()
        weights[np.isnan(weights)] = 1 ## IMPORTANT: make nan weights to 1. to ignore them
        this_var = this_mc_evt_df[var_evt_reco_col]
        this_var = np.clip(this_var, bins[0], bins[-1] - eps)
        this_n, bins = np.histogram(this_var, bins=bins, weights=weights)
        this_signal = this_signal - this_n # a subtracted hitogram for a multiverse
    univ_events.append(this_signal)

for univ_event in univ_events:
    plt.step(bin_edges, np.append(univ_event, univ_event[-1]), where='post', color="gray")
    plt.xlim(bins[0], bins[-1])
    plt.xlabel(var_labels[0])
    plt.ylabel("Events")

plt.step(bin_edges, np.append(nominal_event, nominal_event[-1]), where='post', color="black", label="Nominal")

plt.title("Background Subtracted Universes")
plt.legend()
if save_fig:
    plt.savefig("{}/{}-background_subtracted_universes.pdf".format(save_fig_dir, var_save_name), bbox_inches='tight')
plt.show()

In [39]:
unif_bin = np.linspace(0., float(len(bins) - 1), len(bins))
extent = [unif_bin[0], unif_bin[-1], unif_bin[0], unif_bin[-1]]

x_edges = np.array(bins)
y_edges = np.array(bins)
x_tick_positions = (unif_bin[:-1] + unif_bin[1:]) / 2
y_tick_positions = (unif_bin[:-1] + unif_bin[1:]) / 2

x_labels = bin_range_labels(x_edges)
y_labels = bin_range_labels(y_edges)

def plot_heatmap(matrix, title, save_fig=False, save_fig_name=None):
    plt.imshow(matrix, extent=extent, origin="lower")
    plt.colorbar()
    plt.xticks(x_tick_positions, x_labels, rotation=45, ha="right")
    plt.yticks(y_tick_positions, y_labels)
    plt.xlabel(var_labels[1])
    plt.ylabel(var_labels[1])
    for i in range(matrix.shape[0]):      # rows (y)
        for j in range(matrix.shape[1]):  # columns (x)
            value = matrix[i, j]
            if not np.isnan(value):  # skip NaNs
                plt.text(
                    j + 0.5, i + 0.5,
                    f"{value:.2f}",
                    ha="center", va="center",   
                    color=get_text_color(value),
                    fontsize=10
                )
    plt.title(title)
    if save_fig:
        plt.savefig("{}.pdf".format(save_fig_name), bbox_inches='tight')
    plt.show();

In [ ]:
univ_events = np.array(univ_events)
Flux_Covariance = np.cov(univ_events, rowvar=False)
Flux_Covariance = np.atleast_2d(Flux_Covariance)
print(Flux_Covariance)
plot_heatmap(Flux_Covariance, "Covariance - Flux")

Flux_Covariance_Frac = Flux_Covariance / (nominal_event[:, None] * nominal_event[None, :])
plot_heatmap(Flux_Covariance_Frac, "Fractional Covariance - Flux")

Flux_Correlation = np.corrcoef(univ_events.T)
Flux_Correlation = np.atleast_2d(Flux_Correlation)
plot_heatmap(Flux_Correlation, "Correlation - Flux")

# GENIE

In [ ]:
# nominal_event = fake_data

# w/o background
nom_Signal = Response @ true_signal

GENIE_Covariance_Frac = np.zeros((len(nom_Signal), len(nom_Signal)))
GENIE_Covariance = np.zeros((len(nom_Signal), len(nom_Signal)))

n_univ_genie = 100
for uidx in range(n_univ_genie):
    univ_col_evt = ("GENIE", "univ_{}".format(uidx), "", "", "", "", "", "")
    univ_col_mc = ("GENIE", "univ_{}".format(uidx), "")

    # new response matrix for univ
    reco_vs_true = get_smear_matrix(true_var_signal_sel, var_signal, bins_2d, 
                                    weights=mc_evt_df[mc_evt_df.nuint_categ == 1][univ_col_evt], plot=False)

    true_signal_univ, _ = np.histogram(var_truth_signal, bins=bins, 
                                    weights=weight_truth_signal*mc_nu_df[mc_nu_df.nuint_categ == 1][univ_col_mc])
    eff = get_eff(reco_vs_true, true_signal_univ) # for xsec syst
    # eff = get_eff(reco_vs_true, true_signal) # for flux syst
    univ_Response = get_response_matrix(reco_vs_true, eff, bins, plot=False)

    # signal for univ
    univ_Signal = univ_Response @ true_signal #_univ # TODO: check if this is correct

    # w/ background for univ # TODO: check if this is correct
    # for this_mc_evt_df in mc_evt_df_divided[1:]:
    #     weights = this_mc_evt_df[univ_col_evt].copy()
    #     weights[np.isnan(weights)] = 1 ## IMPORTANT: make nan weights to 1. to ignore them
    #     this_var = this_mc_evt_df[var_evt_reco_col]
    #     this_var = np.clip(this_var, bins[0], bins[-1] - eps)
    #     background_univ, bins = np.histogram(this_var, bins=bins, weights=weights)
    #     univ_Signal += background_univ

    plt.hist(bin_centers, bins=bins, weights=univ_Signal, histtype="step", color="gray")


    for i in range(len(univ_Signal)):
        for j in range(len(univ_Signal)):
            # w/o background
            nom_i = nom_Signal[i] * xsec_unit
            nom_j = nom_Signal[j] * xsec_unit

            # w/ background
            # nom_i = nominal_event[i] * xsec_unit
            # nom_j = nominal_event[j] * xsec_unit

            univ_i = univ_Signal[i] * xsec_unit
            univ_j = univ_Signal[j] * xsec_unit

            frac_cov_entry = ((univ_i - nom_i) / nom_i) * ( (univ_j - nom_j) / nom_j)
            if frac_cov_entry > 0:
                this_frac_cov = max( frac_cov_entry, eps)
            else:
                this_frac_cov = min( frac_cov_entry, eps)

            cov_entry = (univ_i - nom_i) * (univ_j - nom_j)
            if cov_entry > 0:
                this_cov = max( cov_entry, eps)
            else:
                this_cov = min( cov_entry, eps)

            GENIE_Covariance[i, j] += this_cov
            GENIE_Covariance_Frac[i, j] += this_frac_cov

# w/o background
plt.hist(bin_centers, bins=bins, weights=nom_Signal, histtype="step", color="black", label="nominal_event")
# w/ background
# plt.hist(bin_centers, bins=bins, weights=nominal_event, histtype="step", color="black", label="nominal_event")

plt.xlim(bins[0], bins[-1])
plt.title("GENIE")
plt.legend()
plt.show()

GENIE_Covariance_Frac = GENIE_Covariance_Frac / n_univ_genie
GENIE_Covariance = GENIE_Covariance / n_univ_genie

In [ ]:
plot_heatmap(GENIE_Covariance, "Covariance - GENIE")
plot_heatmap(GENIE_Covariance_Frac, "Fractional Covariance - GENIE")

GENIE_Correlation = np.zeros_like(GENIE_Covariance)
for i in range(len(nominal_event)):
    for j in range(len(nominal_event)):
        GENIE_Correlation[i, j] = GENIE_Covariance[i, j] / (np.sqrt(GENIE_Covariance[i, i]) * np.sqrt(GENIE_Covariance[j, j]))
plot_heatmap(GENIE_Correlation, "Correlation - GENIE")

## Total

In [ ]:
Covariance_Frac = Flux_Covariance_Frac + GENIE_Covariance_Frac

Covariance = np.zeros_like(Covariance_Frac)
for i in range(len(nominal_event)):
    for j in range(len(nominal_event)):
        Covariance[i, j] = Covariance_Frac[i, j] * (nom_Signal[i] * nom_Signal[j])

In [ ]:
plot_heatmap(Covariance_Frac, "Total Fractional Covariance")

Singal distribution with error bars from diagonal components of covariance matrix

In [ ]:
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

# Compute bin centers for error bars

frac_uncert = np.sqrt(np.diag(Covariance_Frac))
plt.errorbar(bin_centers, nominal_event, yerr=frac_uncert*nominal_event, fmt='o', color='black', label='Subtracted (syst. error)', capsize=3)
plt.xlim(bins[0], bins[-1])
plt.xlabel(var_labels[0])
plt.ylabel("Events")
plt.legend()

if save_fig:
    plt.savefig("{}/{}-bkg_subtracted_event_rates.pdf".format(save_fig_dir, var_save_name), bbox_inches='tight')
plt.show()

# Unfolding

In [52]:
C_type = 2
Norm_type = 1
unfold = WienerSVD(Response, true_signal, nominal_event, Covariance, C_type, Norm_type)

In [ ]:
unfold['unfold']

In [ ]:
unfold['UnfoldCov']

In [ ]:
Unfold = unfold['unfold']
UnfoldCov = unfold['UnfoldCov']
Unfold_uncert = np.sqrt(np.diag(Covariance))

step_handle, = plt.step(bin_edges, np.append(Unfold, Unfold[-1]), where='post', label='Unfolded')
bar_handle = plt.bar(
    bin_centers,
    2*Unfold_uncert,
    width=(bin_edges[1] - bin_edges[0]) * 1.,
    bottom=Unfold - Unfold_uncert,
    color='gray',
    alpha=0.5,
    linewidth=0,
    label='Unfolded Stat. error (box)'
)
reco_handle, = plt.plot(bin_centers, reco_signal_sel, 'o', label='reco_signal')
true_handle, = plt.plot(bin_centers, true_signal, 'o', label='true_signal')

# Custom order for legend
handles = [step_handle, bar_handle, reco_handle, true_handle]
labels = [
    'Unfolded',
    'Unfolded error',
    'reco_signal',
    'true_signal'
]

plt.legend(handles, labels)
plt.xlim(bins[0], bins[-1])
plt.xlabel(var_labels[0])
# plt.ylim(0., 5000.)
plt.ylabel("Events")

if save_fig:
    plt.savefig("{}/{}-unfolded_event_rates.pdf".format(save_fig_dir, var_save_name), bbox_inches='tight')
plt.show()